In [0]:
%sql
-- Definição do catálogo e schema

USE CATALOG mvp;
USE SCHEMA gold;

In [0]:
%sql
-- Consulta para responder a pergunta 1
-- Quantos voos domésticos foram realizados por mês em 2024?

SELECT mes_voo,
CASE WHEN mes_voo = 1 THEN 'Janeiro'
WHEN mes_voo = 2 THEN 'Fevereiro'
WHEN mes_voo = 3 THEN 'Março'
WHEN mes_voo = 4 THEN 'Abril'
WHEN mes_voo = 5 THEN 'Maio'
WHEN mes_voo = 6 THEN 'Junho'
WHEN mes_voo = 7 THEN 'Julho'
WHEN mes_voo = 8 THEN 'Agosto'
WHEN mes_voo = 9 THEN 'Setembro'
WHEN mes_voo = 10 THEN 'Outubro'
WHEN mes_voo = 11 THEN 'Novembro'
WHEN mes_voo = 12 THEN 'Dezembro'
END AS mes_nome,
COUNT(*) AS quantidade
FROM mvp.gold.voos_flat
WHERE situacao_voo = 'REALIZADO'
GROUP BY mes_voo
ORDER BY quantidade DESC;

mes_voo,mes_nome,quantidade
7,Julho,69225
8,Agosto,66736
1,Janeiro,66714
12,Dezembro,66637
10,Outubro,66057
9,Setembro,64165
4,Abril,64050
3,Março,63206
11,Novembro,62398
6,Junho,61944


In [0]:
%sql
-- Consulta para responder a pergunta 2
-- Quais aeroportos concentraram o maior volume de partidas?

SELECT aeroporto_origem_nome,
COUNT(*) AS quantidade,
aeroporto_origem_municipio,
aeroporto_origem_uf
FROM mvp.gold.voos_flat
WHERE situacao_voo = 'REALIZADO'
GROUP BY aeroporto_origem_nome, aeroporto_origem_municipio, aeroporto_origem_uf
ORDER BY quantidade DESC
LIMIT 10;

aeroporto_origem_nome,quantidade,aeroporto_origem_municipio,aeroporto_origem_uf
São Paulo/Congonhas - Deputado Freitas Nobre,94262,SÃO PAULO,São Paulo
Guarulhos - Governador André Franco Montoro,92093,GUARULHOS,São Paulo
Tancredo Neves,53371,CONFINS,Minas Gerais
Viracopos,53078,CAMPINAS,São Paulo
Presidente Juscelino Kubitschek,50962,BRASÍLIA,Distrito Federal
AEROPORTO INTERNACIONAL RECIFE/GUARARAPES - GILBERTO FREYRE,39129,RECIFE,Pernambuco
RIO DE JANEIRO/GALEÃO - ANTÔNIO CARLOS JOBIM,34749,RIO DE JANEIRO,Rio de Janeiro
Santos Dumont,28619,RIO DE JANEIRO,Rio de Janeiro
DEPUTADO LUÍS EDUARDO MAGALHÃES,25553,SALVADOR,Bahia
Afonso Pena,24127,CURITIBA,Paraná


In [0]:
%sql
-- Consulta para responder a pergunta 3
-- Quais aeroportos concentraram o maior volume de chegadas?

SELECT aeroporto_destino_nome,
COUNT(*) AS quantidade,
aeroporto_destino_municipio,
aeroporto_destino_uf
FROM mvp.gold.voos_flat
WHERE situacao_voo = 'REALIZADO'
GROUP BY aeroporto_destino_nome, aeroporto_destino_municipio, aeroporto_destino_uf
ORDER BY quantidade DESC
LIMIT 10;

aeroporto_destino_nome,quantidade,aeroporto_destino_municipio,aeroporto_destino_uf
São Paulo/Congonhas - Deputado Freitas Nobre,94249,SÃO PAULO,São Paulo
Guarulhos - Governador André Franco Montoro,92137,GUARULHOS,São Paulo
Tancredo Neves,53368,CONFINS,Minas Gerais
Viracopos,53054,CAMPINAS,São Paulo
Presidente Juscelino Kubitschek,50949,BRASÍLIA,Distrito Federal
AEROPORTO INTERNACIONAL RECIFE/GUARARAPES - GILBERTO FREYRE,39153,RECIFE,Pernambuco
RIO DE JANEIRO/GALEÃO - ANTÔNIO CARLOS JOBIM,34654,RIO DE JANEIRO,Rio de Janeiro
Santos Dumont,28583,RIO DE JANEIRO,Rio de Janeiro
DEPUTADO LUÍS EDUARDO MAGALHÃES,25562,SALVADOR,Bahia
Afonso Pena,24152,CURITIBA,Paraná


In [0]:
%sql
-- Consulta para responder a pergunta 4
-- Qual a participação percentual dos 10 maiores aeroportos no total de voos realizados em 2024?

WITH base AS (
    SELECT aeroporto_origem_nome AS aeroporto,
    aeroporto_origem_municipio AS municipio,
    aeroporto_origem_uf AS uf
    FROM mvp.gold.voos_flat
    WHERE situacao_voo = 'REALIZADO'

    UNION ALL

    SELECT aeroporto_destino_nome AS aeroporto,
    aeroporto_destino_municipio AS municipio,
    aeroporto_destino_uf AS uf
    FROM mvp.gold.voos_flat
    WHERE situacao_voo = 'REALIZADO'
),

total_voos AS (
    SELECT COUNT(*) AS total
    FROM base
),

ranking AS (
    SELECT 
        aeroporto,
        municipio,
        uf,
        COUNT(*) AS quantidade
    FROM base
    GROUP BY aeroporto, municipio, uf
)

SELECT 
    r.aeroporto,
    r.municipio,
    r.uf,
    r.quantidade,
    ROUND((r.quantidade / t.total) * 100, 2) AS percentual_participacao
FROM ranking r
CROSS JOIN total_voos t
ORDER BY r.quantidade DESC
LIMIT 10;

aeroporto,municipio,uf,quantidade,percentual_participacao
São Paulo/Congonhas - Deputado Freitas Nobre,SÃO PAULO,São Paulo,188511,12.2
Guarulhos - Governador André Franco Montoro,GUARULHOS,São Paulo,184230,11.93
Tancredo Neves,CONFINS,Minas Gerais,106739,6.91
Viracopos,CAMPINAS,São Paulo,106132,6.87
Presidente Juscelino Kubitschek,BRASÍLIA,Distrito Federal,101911,6.6
AEROPORTO INTERNACIONAL RECIFE/GUARARAPES - GILBERTO FREYRE,RECIFE,Pernambuco,78282,5.07
RIO DE JANEIRO/GALEÃO - ANTÔNIO CARLOS JOBIM,RIO DE JANEIRO,Rio de Janeiro,69403,4.49
Santos Dumont,RIO DE JANEIRO,Rio de Janeiro,57202,3.7
DEPUTADO LUÍS EDUARDO MAGALHÃES,SALVADOR,Bahia,51115,3.31
Afonso Pena,CURITIBA,Paraná,48279,3.13


In [0]:
%sql
-- Consulta para responder a pergunta 5
-- Como o volume mensal de vôos variou ao longo do ano?

WITH dados AS (
    SELECT 
        mes_voo,
        COUNT(*) AS quantidade
    FROM mvp.gold.voos_flat
    WHERE situacao_voo = 'REALIZADO'
    GROUP BY mes_voo
),
ordenado AS (
    SELECT *
    FROM dados
    ORDER BY mes_voo
)
SELECT 
    mes_voo,
    quantidade,
    ROUND( (quantidade - LAG(quantidade) OVER (ORDER BY mes_voo)) 
        / LAG(quantidade) OVER (ORDER BY mes_voo) * 100, 2 ) AS variacao_mensal
FROM ordenado;


mes_voo,quantidade,variacao_mensal
1,66714,null
2,59561,-10.72
3,63206,6.12
4,64050,1.34
5,61719,-3.64
6,61944,0.36
7,69225,11.75
8,66736,-3.6
9,64165,-3.85
10,66057,2.95


In [0]:
%sql
-- Consulta para responder a pergunta 6
-- Quais estações do ano apresentam maior e menor movimento aéreo?

SELECT estacao_ano,
COUNT(*) AS quantidade
FROM mvp.gold.voos_flat
WHERE situacao_voo == 'REALIZADO'
GROUP BY estacao_ano
ORDER BY quantidade DESC;

estacao_ano,quantidade
Inverno,197905
Verao,192912
Primavera,192620
Outono,188975


In [0]:
%sql
-- Consulta para responder a pergunta 7
-- Quais pares origem–destino tiveram maior volume de voos?

SELECT aeroporto_origem_nome AS origem,
aeroporto_origem_uf AS uf_origem,
aeroporto_destino_nome AS destino,
aeroporto_destino_uf AS uf_destino,
COUNT(*) AS qtde_voos
FROM mvp.gold.voos_flat
WHERE situacao_voo = 'REALIZADO'
GROUP BY origem, destino, uf_origem, uf_destino
ORDER BY qtde_voos DESC
LIMIT 20;

origem,uf_origem,destino,uf_destino,qtde_voos
São Paulo/Congonhas - Deputado Freitas Nobre,São Paulo,Santos Dumont,Rio de Janeiro,18360
Santos Dumont,Rio de Janeiro,São Paulo/Congonhas - Deputado Freitas Nobre,São Paulo,18356
São Paulo/Congonhas - Deputado Freitas Nobre,São Paulo,Presidente Juscelino Kubitschek,Distrito Federal,9474
Presidente Juscelino Kubitschek,Distrito Federal,São Paulo/Congonhas - Deputado Freitas Nobre,São Paulo,9423
Tancredo Neves,Minas Gerais,São Paulo/Congonhas - Deputado Freitas Nobre,São Paulo,8795
São Paulo/Congonhas - Deputado Freitas Nobre,São Paulo,Tancredo Neves,Minas Gerais,8781
São Paulo/Congonhas - Deputado Freitas Nobre,São Paulo,Afonso Pena,Paraná,7249
Afonso Pena,Paraná,São Paulo/Congonhas - Deputado Freitas Nobre,São Paulo,7232
São Paulo/Congonhas - Deputado Freitas Nobre,São Paulo,Hercílio Luz,Santa Catarina,5562
Guarulhos - Governador André Franco Montoro,São Paulo,AEROPORTO INTERNACIONAL RECIFE/GUARARAPES - GILBERTO FREYRE,Pernambuco,5539


In [0]:
%sql
-- Consulta para responder a pergunta 8
-- Quais cidades apresentam maior diversidade de destinos atendidos?

SELECT aeroporto_origem_municipio AS cidade,
aeroporto_origem_uf AS uf,
COUNT(DISTINCT aeroporto_destino_municipio) AS qtde_destinos
FROM mvp.gold.voos_flat
WHERE situacao_voo = 'REALIZADO'
GROUP BY cidade, uf
ORDER BY qtde_destinos DESC
LIMIT 10;

cidade,uf,qtde_destinos
CAMPINAS,São Paulo,78
CONFINS,Minas Gerais,74
GUARULHOS,São Paulo,64
SÃO PAULO,São Paulo,57
RECIFE,Pernambuco,54
BRASÍLIA,Distrito Federal,51
RIO DE JANEIRO,Rio de Janeiro,48
FORTALEZA,Ceará,45
CURITIBA,Paraná,43
SALVADOR,Bahia,43


In [0]:
%sql
-- Consulta para responder a pergunta 9
-- Quais companhias aéreas operaram o maior número de voos domésticos?

SELECT nome_companhia AS companhia,
COUNT(*) AS qtde_voos
FROM mvp.gold.voos_flat
WHERE situacao_voo = 'REALIZADO'
GROUP BY companhia
ORDER BY qtde_voos DESC
LIMIT 5;

companhia,qtde_voos
Azul Linhas Aéreas,290158
LATAM Airlines,240808
Gol Linhas Aéreas,199242
Azul Conecta,20560
Passaredo Transportes Aéreos,15602


In [0]:
%sql
-- Consulta para responder a pergunta 10
-- Existe concentração da operação em poucas companhias?

WITH total AS (
    SELECT COUNT(*) AS total_voos
    FROM mvp.gold.voos_flat
    WHERE situacao_voo = 'REALIZADO'
)
SELECT
    nome_companhia,
    COUNT(*) AS qtde_voos,
    ROUND( (COUNT(*) / total_voos) * 100, 2 ) AS participacao_pct
FROM mvp.gold.voos_flat, total
WHERE situacao_voo = 'REALIZADO'
GROUP BY nome_companhia, total_voos
ORDER BY qtde_voos DESC
LIMIT 3;

nome_companhia,qtde_voos,participacao_pct
Azul Linhas Aéreas,290158,37.57
LATAM Airlines,240808,31.18
Gol Linhas Aéreas,199242,25.79
